<h1 align="center"> Aplicações em Processamento de Linguagem Natural </h1>
<h2 align="center"> Aula 03 - Técnicas de Pré-Processamento de Texto e Similaridade</h2>
<h3 align="center"> Prof. Fernando Vieira da Silva MSc.</h3>

# Integrantes:
### Leandro Freitas 	    - RA: 080162
### Paulo Érico de Freitas	- RA: 183545
### Renan Souza		        - RA: 070487 

<p><b>Exercício 3:</b>

Escreva um chatbot que, dado uma pergunta em Inglês, encontre uma pergunta mais parecida no corpus de perguntas e respostas disponível no Kaggle (https://www.kaggle.com/rtatman/questionanswer-dataset#S08_question_answer_pairs.txt) e exiba a resposta.

In [ ]:
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize

In [ ]:
#file = open('../input/S08_question_answer_pairs.txt', 'r')
file = open('../input/questionanswer-dataset/S08_question_answer_pairs.txt', 'r')
array_arquivo = []
for line in file:      
    array_arquivo.append(line.strip().split('\t'))
file = open('../input/questionanswer-dataset/S09_question_answer_pairs.txt', 'r')
#array_arquivo = []
for line in file:      
    array_arquivo.append(line.strip().split('\t'))
file = open('../input/questionanswer-dataset/S10_question_answer_pairs.txt', 'r', encoding = "ISO-8859-1")
#array_arquivo = []
for line in file:      
    array_arquivo.append(line.strip().split('\t'))
#import pandas as pd
#import numpy as np
#import string
#df_S08 = pd.read_csv('../input/S08_question_answer_pairs.txt', sep="\t", header=0)
#df_S09 = pd.read_csv('../input/S09_question_answer_pairs.txt', sep="\t", header=0)
#df_S10 = pd.read_csv('../input/S10_question_answer_pairs.txt', sep="\t", header=0, encoding = "ISO-8859-1")

In [ ]:
texto = ''
for a in array_arquivo:
    texto += a[1] + ' '

In [ ]:
array_arquivo[1][2]

In [ ]:
sents_token = sent_tokenize(texto)
print(sents_token)

In [ ]:
sents_array = np.array(sents_token)
print(sents_array[10])

In [ ]:
a = pd.DataFrame(sents_array)
len(a)
a.dropna()
len(a)

In [ ]:
[print(a) for a in sents_array]

In [ ]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

stopwords_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
def my_tokenizer(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)

tfs = tfidf_vectorizer.fit_transform(sents_array)

print(tfs.shape)
print(tfs)

In [ ]:
print([k for k in tfidf_vectorizer.vocabulary_.keys()][:20])

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd_transformer = TruncatedSVD(n_components=1000)

svd_transformer.fit(tfs)

print(sorted(svd_transformer.explained_variance_ratio_)[::-1][:30])

In [ ]:
cummulative_variance = 0.0
k = 0
for var in sorted(svd_transformer.explained_variance_ratio_)[::-1]:
    cummulative_variance += var
    if cummulative_variance >= 0.5:
        break
    else:
        k += 1
        
print(k)

In [ ]:
svd_transformer = TruncatedSVD(n_components=k)
svd_data = svd_transformer.fit_transform(tfs)
print(sorted(svd_transformer.explained_variance_ratio_)[::-1])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sents_array)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def retornar_pergunta(index):    
    return array_arquivo[index + 1][1]
def retornar_resposta(index):    
    if array_arquivo[index + 1][2] == 'NULL':
        return 'We didn t find an answer, could you ask another question?'
    else:
        return array_arquivo[index + 1][2]

In [ ]:
#Escreva uma pergunta 
#query_vect = tfidf_vectorizer.transform(["When did Jão graduate from William and Mary?"])
# ou Escolha uma pergunta dentre as existentes
query_vect = tfidf_vectorizer.transform([sents_array[3000]])
#789
#777

result_cosine = cosine_similarity(query_vect, tfidf_matrix)

print(np.amax(result_cosine))
print(np.argmax(result_cosine))
print(retornar_pergunta(np.argmax(result_cosine)))
print(retornar_resposta(np.argmax(result_cosine)))